In [1]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import  KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.feature_selection import RFE
import operator
import seaborn as sn
from scipy import stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_predict
from imblearn.over_sampling import SVMSMOTE
from sklearn.model_selection import cross_validate
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import BernoulliNB
import pandasql as ps
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [464]:
df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/TRAIN_CLEARED_2part.csv")
df = df.drop(['Unnamed: 0'], axis=1)

In [232]:
df = df.drop(['NewName', 'ClubInvolved', 'Club', 'ClubAge', 'ClubNumPlayersUsedPerMatch',\
     'ClubGoalPerShotOnTargetPerMatch', 'ClubShotOnTargetAgainstUsedPerMatch', 'ClubPointsPerMatch', 'Fee'], axis = 1)

In [465]:
df.columns

Index(['NewName', 'ClubInvolved', 'League', 'MainPosition', 'LeagueInvolved',
       'CorrMarketValue', 'Country', 'Club', 'Position', 'Foot',
       'IntReputation', 'Heading_accuracy', 'Reactions', 'Balance', 'Jumping',
       'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL',
       'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'Fee', 'ClubRank',
       'ClubInvolvedAge', 'ClubAge', 'ClubInvolvedCleanSheetsRate',
       'MinutesPlayedPer90', 'ContractLeft', 'Pace_main', 'Shooting_main',
       'Defending_main', 'Passing_main', 'Physical_main', 'Age',
       'ClubInvolvedPointsPerMatch', 'ClubPointsPerMatch',
       'ClubInvolvedNumPlayersUsedPerMatch',
       'ClubInvolvedGoalPerShotOnTargetPerMatch', 'ClubNumPlayersUsedPerMatch',
       'ClubGoalPerShotOnTargetPerMatch',
       'ClubInvolvedShotOnTargetAgainstUsedPerMatch',
       'ClubShotOnTargetAgainstUsedPerMatch', 'GoalsPerMinutesPlayed',
       'AssistsPerMinutesPlayed', 'YellowCardsPerMinutesPlayed',
       'RedCards

In [469]:
avg_df_club = pd.DataFrame(columns=['Club', 'ClubRank', 'ClubPointsPerMatch',\
        'ClubNumPlayersUsedPerMatch', 'ClubGoalPerShotOnTargetPerMatch', 'ClubShotOnTargetAgainstUsedPerMatch',\
            'Season'])
for year in range(2008, 2022):
    df_rank = df[(df.Season >= year - 2) & (df.Season <= year) ][['Club', 'ClubRank', 'ClubPointsPerMatch',\
        'ClubNumPlayersUsedPerMatch', 'ClubGoalPerShotOnTargetPerMatch', 'ClubShotOnTargetAgainstUsedPerMatch',\
            'Season']].drop_duplicates()
    df_rank = df_rank.groupby('Club').mean(['ClubRank', 'ClubPointsPerMatch',\
        'ClubNumPlayersUsedPerMatch', 'ClubGoalPerShotOnTargetPerMatch', 'ClubShotOnTargetAgainstUsedPerMatch',\
            'Season']).reset_index()
    df_rank['Season'] = year
    avg_df_club = avg_df_club.append(df_rank, ignore_index=True)
avg_df_club[avg_df_club.Club == 'FC Barcelona'].head()

,Club,ClubRank,ClubPointsPerMatch,ClubNumPlayersUsedPerMatch,ClubGoalPerShotOnTargetPerMatch,ClubShotOnTargetAgainstUsedPerMatch,Season
59,FC Barcelona,2.000000,2.000000,0.578947,0.007105,3.473684,2008
271,FC Barcelona,2.500000,1.881579,0.657895,0.007632,3.697368,2009
502,FC Barcelona,2.000000,2.017544,0.692982,0.008070,3.561404,2010
740,FC Barcelona,1.666667,2.219298,0.710526,0.008333,3.421053,2011
979,FC Barcelona,1.000000,2.473684,0.710526,0.008333,3.131579,2012


In [467]:
avg_df = pd.DataFrame(columns=['ClubInvolved', 'ClubInvolvedCleanSheetsRate', \
        'ClubInvolvedPointsPerMatch', 'ClubInvolvedGoalPerShotOnTargetPerMatch', 'ClubInvolvedShotOnTargetAgainstUsedPerMatch',\
            'Season'])
for year in range(2008, 2022):
    df_rank = df[(df.Season >= year - 2) & (df.Season <= year) ][['ClubInvolved', 'ClubInvolvedCleanSheetsRate',\
        'ClubInvolvedPointsPerMatch', 'ClubInvolvedGoalPerShotOnTargetPerMatch', 'ClubInvolvedShotOnTargetAgainstUsedPerMatch']].drop_duplicates()
    df_rank = df_rank.groupby('ClubInvolved').mean(['ClubInvolvedCleanSheetsRate'\
        'ClubInvolvedPointsPerMatch', 'ClubInvolvedGoalPerShotOnTargetPerMatch', 'ClubInvolvedShotOnTargetAgainstUsedPerMatch',\
            ]).reset_index()
    df_rank['Season'] = year
    avg_df = avg_df.append(df_rank, ignore_index=True)
avg_df[avg_df.ClubInvolved == 'FC Barcelona'].head()

,ClubInvolved,ClubInvolvedCleanSheetsRate,ClubInvolvedPointsPerMatch,ClubInvolvedGoalPerShotOnTargetPerMatch,ClubInvolvedShotOnTargetAgainstUsedPerMatch,Season
123,FC Barcelona,39.500000,2.000000,0.007105,3.473684,2008
581,FC Barcelona,38.150000,1.881579,0.007632,3.697368,2009
1191,FC Barcelona,38.600000,2.017544,0.008070,3.561404,2010
1890,FC Barcelona,42.966667,2.219298,0.008333,3.421053,2011
2589,FC Barcelona,47.366667,2.473684,0.008333,3.131579,2012


In [470]:
#join tables with train df
q1 = """SELECT DF.*,
        i.ClubInvolvedCleanSheetsRate ClubInvolvedCleanSheetsRateAvg,
        i.ClubInvolvedGoalPerShotOnTargetPerMatch ClubInvolvedGoalPerShotOnTargetPerMatchAvg,
        i.ClubInvolvedShotOnTargetAgainstUsedPerMatch ClubInvolvedShotOnTargetAgainstUsedPerMatchAvg,
        i.ClubInvolvedPointsPerMatch ClubInvolvedPointsPerMatchAvg,
        a.ClubRank ClubRankAvg,
        a.ClubPointsPerMatch ClubPointsPerMatchAvg,
        a.ClubNumPlayersUsedPerMatch ClubNumPlayersUsedPerMatchíAvg,
        a.ClubGoalPerShotOnTargetPerMatch ClubGoalPerShotOnTargetPerMatchAvg,
        a.ClubShotOnTargetAgainstUsedPerMatch ClubShotOnTargetAgainstUsedPerMatchAvg


        FROM df DF 

        LEFT JOIN avg_df i
        ON DF.ClubInvolved = i.ClubInvolved
        AND DF.Season = i.Season

        LEFT JOIN avg_df_club a
        ON DF.Club = a.Club
        AND DF.Season = a.Season
        """

df = ps.sqldf(q1, locals())

df = df.drop_duplicates()

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11013 entries, 0 to 11027
Data columns (total 58 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   NewName                                         11013 non-null  object 
 1   ClubInvolved                                    11013 non-null  object 
 2   League                                          11013 non-null  object 
 3   MainPosition                                    11013 non-null  object 
 4   LeagueInvolved                                  11013 non-null  object 
 5   CorrMarketValue                                 11013 non-null  float64
 6   Country                                         11013 non-null  object 
 7   Club                                            11013 non-null  object 
 8   Position                                        11013 non-null  object 
 9   Foot                                   

In [471]:
df = df.drop(['NewName', 'Fee', 'Club'], axis = 1)

In [472]:
#first 10
df['target'] = 0
df.loc[df.League == 'Spain Primera Division', 'target'] = 1
df.loc[df.ClubRank > 10, 'target'] = 0

In [473]:
df = df.drop(['League'], axis=1)

In [474]:
df['target'].value_counts()

0    10341
1      672
Name: target, dtype: int64

In [475]:
#set feature list and target
feature_list = list(df.columns.values)
feature_list.remove('target')
target = 'target'

# define dataset
X = df[feature_list]
y = df[target]

In [215]:
import category_encoders as ce
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

In [216]:
#Impute missing values
imputer = IterativeImputer()
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [217]:
#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

In [218]:
X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']
# X_imp_st['Foot_Reactions'] = X_imp_st['Foot'] + X_imp_st['Reactions']

In [36]:
from sklearn.model_selection import RepeatedStratifiedKFold
# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

[14:53:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:53:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:53:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:53:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[14:53:32] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:53:32] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:53:32] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:53:32] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarnin

...

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[14:54:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:54:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:54:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:54:02] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 9.752612638473511
score_time 0.05555117130279541
test_acc 0.9616888768957127
test_recall 0.46740561896400357
test_auc 0.9619072467076053
test_precison 0.8299329458665141
test_f1 0.5961996154982265


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Different imputers

In [37]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RepeatedStratifiedKFold

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator = DecisionTreeRegressor(max_features="sqrt", random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

[14:56:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:56:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:56:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:56:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarnin

[14:56:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[14:57:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:57:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:57:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 12.879509878158569
score_time 0.0643926978111267
test_acc 0.9588324121806064
test_recall 0.4159240561896401
test_auc 0.9379930956428266
test_precison 0.821649655300005
test_f1 0.5504366524532004


In [38]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  KNeighborsRegressor(n_neighbors=15))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

[15:04:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:04:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:04:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:04:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[15:04:55] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:04:55] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:04:55] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:04:56] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

...

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:05:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:05:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:05:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 9.348097217082977
score_time 0.04795668125152588
test_acc 0.9623686760904514
test_recall 0.46659350307287106
test_auc 0.9620685848981072
test_precison 0.8477189167850103
test_f1 0.5996248191867094


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

In [39]:
from sklearn.ensemble import HistGradientBoostingRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  HistGradientBoostingRegressor(random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

[15:15:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:15:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:15:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:15:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[15:15:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:15:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:15:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:15:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

...

[15:15:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:15:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:15:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:15:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 8.353373074531556
score_time 0.04689544439315796
test_acc 0.9608271370112531
test_recall 0.44284460052677793
test_auc 0.9545891837392773
test_precison 0.8397217092834353
test_f1 0.5774713005208215


In [40]:

from sklearn.ensemble import GradientBoostingRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  LGBMRegressor(random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

[15:19:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[15:19:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

...

[15:19:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:32] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:32] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:32] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[15:19:40] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:40] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:40] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:40] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 8.338951349258423
score_time 0.03517661094665527
test_acc 0.9604189942295636
test_recall 0.4360294117647059
test_auc 0.9572484605677338
test_precison 0.8354763068818654
test_f1 0.5719715684138631


In [41]:

from sklearn.ensemble import GradientBoostingRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  XGBRegressor(random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

[15:45:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:45:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:45:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:45:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[15:45:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:45:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:45:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:45:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

...

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:46:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:46:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:46:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


fit_time 7.292539191246033
score_time 0.04140472412109375
test_acc 0.9598755579980682
test_recall 0.43237489025460923
test_auc 0.9541696576087919
test_precison 0.827891097583378
test_f1 0.5657054076043591


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

DecisionTreeRegressor hyperparameter tuning

In [42]:
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer()
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 10.790875720977784
score_time 0.06719082593917847
test_acc 0.9616888768957127
test_recall 0.46740561896400357
test_auc 0.9619072467076053
test_precison 0.8299329458665141
test_f1 0.5961996154982265


XGBoost hyperparameter tuning

In [476]:
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator = DecisionTreeRegressor(max_depth= None, max_features="sqrt", random_state=1, \
    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.2, min_impurity_decrease=0.))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['LeagueInvolved_CorrMarketValue'] = X_imp_st['LeagueInvolved'] * X_imp_st['CorrMarketValue']
X_imp_st['LeagueInvolved_MinutesPlayedPer90'] = X_imp_st['LeagueInvolved'] * X_imp_st['MinutesPlayedPer90']
X_imp_st['LeagueInvolved_Season'] = X_imp_st['LeagueInvolved'] / X_imp_st['Season']
X_imp_st['Season_SP_LA_LIGA'] = X_imp_st['Season'] / X_imp_st['SP_LA_LIGA']



In [57]:
# define the model with default hyperparameters
model = XGBClassifier()
# define the grid of values to search
grid = dict()
grid['verbosity'] = [0]
grid['random_state'] = [1]
grid['n_estimators'] = [300] #100, 500
grid['learning_rate'] = [0.3] #0.01, 0.1, 0.2 , 0.3, 0.4, 0.5
grid['subsample'] = [1] #0.5, 0.75
grid['max_features'] = [10, 20, 30]
grid['max_depth'] = [6] #3,9
#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit='f1')
# execute the grid search
grid_result = grid_search.fit(X_imp_st, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

Best: 0.652790 using {'learning_rate': 0.3, 'max_depth': 6, 'max_features': 10, 'n_estimators': 300, 'random_state': 1, 'subsample': 1, 'verbosity': 0}


In [59]:
#create df for metrics
columns= ['params', 'mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']
indexes = [x for x in range(len(grid_result.cv_results_['params']))]
metric_df = pd.DataFrame(index = indexes, columns=columns)
metric_df = metric_df.fillna(0)
metric_df['params'] = metric_df['params'].astype(str)
metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']] = metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']].astype(float)

metric_df['params'] = grid_result.cv_results_['params']
metric_df['mean_test_acc'] = grid_result.cv_results_['mean_test_acc']
metric_df['mean_test_recall'] = grid_result.cv_results_['mean_test_recall']
metric_df['mean_test_auc'] = grid_result.cv_results_['mean_test_auc']
metric_df['mean_test_precison'] = grid_result.cv_results_['mean_test_precison']
metric_df['mean_test_f1'] = grid_result.cv_results_['mean_test_f1']

metric_df.sort_values(by='mean_test_f1', ascending=False)

,params,mean_test_acc,mean_test_auc,mean_test_recall,mean_test_precison,mean_test_f1
0,"{'learning_rate': 0.3, 'max_depth': 6, 'max_features': 10, 'n_estimators': 300, 'random_state': 1, 'subsample': 1, 'verbosity': 0}",0.966087,0.967514,0.526888,0.863882,0.65279
1,"{'learning_rate': 0.3, 'max_depth': 6, 'max_features': 20, 'n_estimators': 300, 'random_state': 1, 'subsample': 1, 'verbosity': 0}",0.966087,0.967514,0.526888,0.863882,0.65279
2,"{'learning_rate': 0.3, 'max_depth': 6, 'max_features': 30, 'n_estimators': 300, 'random_state': 1, 'subsample': 1, 'verbosity': 0}",0.966087,0.967514,0.526888,0.863882,0.65279


In [238]:
pd.set_option('display.max_colwidth', None)

In [477]:
# define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=300, random_state=1,\
    subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 50.073533952236176
score_time 0.09237771034240723
test_acc 0.9795691427196196
test_recall 0.7090978928884987
test_auc 0.9865005757308069
test_precison 0.9416994808266118
test_f1 0.807553576070217


In [ ]:
fit_time 57.34267292022705
score_time 0.13555482625961304
test_acc 0.9630488455013797
test_recall 0.48368086040386304
test_auc 0.9653747991061424
test_precison 0.8435213767604435
test_f1 0.6131923937152066

In [295]:
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=300, random_state=1,\
    subsample=1, verbosity=0)
m = model.fit(X_imp_st , y)
zip_iterator = zip(list(X_imp_st.columns), m.feature_importances_)
feature_dict = dict(zip_iterator)
print(dict( sorted(feature_dict.items(), key=operator.itemgetter(1),reverse=True)))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


{'SP_LA_LIGA': 0.12799306, 'Country': 0.056681935, 'ClubPointsPerMatch': 0.0501863, 'ClubNumPlayersUsedPerMatch': 0.032214366, 'FR_LIGUE_1': 0.03047347, 'ClubAge': 0.030179381, 'IT_SERIE_A': 0.028681776, 'ClubGoalPerShotOnTargetPerMatch': 0.027409961, 'LeagueInvolved': 0.02687962, 'CorrMarketValue': 0.026630724, 'ClubInvolved': 0.025608465, 'DEGREE': 0.023303285, 'Foot': 0.023280514, 'GE_BUNDESLIGA': 0.023132382, 'LeagueInvolved_CorrMarketValue': 0.021786347, 'YellowCardsPerMinutesPlayed': 0.021554992, 'Season': 0.02100397, 'ClubShotOnTargetAgainstUsedPerMatch': 0.020598961, 'MainPosition': 0.019029438, 'Reactions': 0.01846348, 'LeagueInvolved_MinutesPlayedPer90': 0.016427264, 'MinutesPlayedPer90': 0.016289752, 'Passing_main': 0.015319379, 'ClubInvolvedAge': 0.015283745, 'Season_SP_LA_LIGA': 0.014877707, 'ClubInvolvedPointsPerMatch': 0.014728617, 'Position': 0.01449454, 'Physical_main': 0.013929144, 'LeagueInvolved_Season': 0.013814685, 'ClubInvolvedShotOnTargetAgainstUsedPerMatch': 0.

Stacking

In [240]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import BernoulliNB

In [460]:
level0 = list()
level0.append(('bernoulli', BernoulliNB(alpha=0.5, binarize=1)))
# level0.append(('knn',  KNeighborsClassifier(n_neighbors=5, weights='distance')))
level0.append(('bagging', BaggingClassifier(n_estimators=100)))
# level0.append(('dectree', DecisionTreeClassifier()))
level0.append(('svc', SVC(C=10, gamma='scale', kernel='rbf')))
level0.append(('gaussian', GaussianNB()))
level0.append(('ada', AdaBoostClassifier(learning_rate=0.7, n_estimators=100,)))
level0.append(('gradient', GradientBoostingClassifier(learning_rate=0.5, max_depth=13, n_estimators=100, random_state=1)))
level0.append(('xgb', XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)))

# define meta learner model
level1 = LogisticRegression()
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

In [461]:
# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 821.1297479748725
score_time 3.9498387694358827
test_acc 0.9487218080371467
test_recall 0.22333187006145744
test_auc 0.9214335522517759
test_precison 0.7802943516178811
test_f1 0.34480980822838597


fit_time 679.1351370096206
score_time 3.4601255774497988
test_acc 0.9508980622061923
test_recall 0.260513608428446
test_auc 0.9100802079079171
test_precison 0.8015705852343018
test_f1 0.39066306746164403

Syntetic data generalization

In [361]:
X_train = X_imp_st.copy()
X_train['target'] = y

positives = X_train[X_train.target == 1]

In [362]:
from sdv.tabular import GaussianCopula
model = GaussianCopula()
model.fit(positives)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:621: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:5003: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2494: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:4994: RuntimeWarning: divide by zero encountered in power
  return cd2*x**(c-1)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress

In [363]:
sample = model.sample(2000)

In [364]:
from sdv.evaluation import evaluate
evaluate(sample, positives, metrics=['CSTest', 'KSTest'], aggregate=False)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,CSTest,Chi-Squared,NaN,NaN,0.0,1.0,MAXIMIZE,"Cannot find fields of types ('boolean', 'categorical')"
1,KSTest,Inverted Kolmogorov-Smirnov D statistic,0.871369,0.871369,0.0,1.0,MAXIMIZE,None


In [365]:
X_synt = X_train.append(sample)

In [366]:
X_synt['target'].value_counts()

0    10356
1     2672
Name: target, dtype: int64

In [367]:
y_synt = X_synt['target']
X_synt = X_synt.drop('target', axis = 1)

In [127]:
# define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=300, random_state=1,\
    subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_synt , y_synt, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 86.19455317258834
score_time 0.12001749277114868
test_acc 0.9753868760431244
test_recall 0.9462272089326078
test_auc 0.9954995103901384
test_precison 0.983655973960284
test_f1 0.964535585244849


SMOTE

In [511]:
from imblearn.over_sampling import SVMSMOTE
#overfit the target label
over = SVMSMOTE(sampling_strategy=0.25)
X_res, y_res = over.fit_resample(X_imp_st, y)

In [188]:
# define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_res, y_res, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 93.83350163698196
score_time 0.11891800165176392
test_acc 0.9423538700064176
test_recall 0.8069992031872509
test_auc 0.9677928738253107
test_precison 0.8948405181078526
test_f1 0.8483844139579133


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [135]:
predict_df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/predict_df_temp3.csv")
predict_df = predict_df.drop('Unnamed: 0', axis = 1)

In [147]:
predict_df[(predict_df.LeagueInvolved == 'Spain Primera Division') & (predict_df.ClubInvolvedRank < 11)][['ClubInvolved', 'ClubInvolvedRank']].drop_duplicates()

,ClubInvolved,ClubInvolvedRank
0,FC Barcelona,3.0
2,Atlético Madrid,1.0
14,Real Madrid CF,2.0
60,Real Sociedad,5.0
84,Villarreal CF,7.0
124,RC Celta de Vigo,8.0
136,Sevilla FC,4.0
157,Real Betis Balompié,6.0
195,Athletic Club de Bilbao,10.0
300,"Atlético Madrid Jun 30,",1.0


In [564]:
predict_df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/predict_df_temp3.csv")
predict_df = predict_df.drop('Unnamed: 0', axis = 1)

In [481]:
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13102 entries, 0 to 13101
Columns: 123 entries, Name to Season.1
dtypes: float64(63), int64(47), object(13)
memory usage: 12.3+ MB


In [565]:
#join tables with train df
q1 = """SELECT DF.*,
        i.ClubInvolvedCleanSheetsRate ClubInvolvedCleanSheetsRateAvg,
        i.ClubInvolvedGoalPerShotOnTargetPerMatch ClubInvolvedGoalPerShotOnTargetPerMatchAvg,
        i.ClubInvolvedShotOnTargetAgainstUsedPerMatch ClubInvolvedShotOnTargetAgainstUsedPerMatchAvg,
        i.ClubInvolvedPointsPerMatch ClubInvolvedPointsPerMatchAvg,
        a.ClubRank ClubRankAvg,
        a.ClubPointsPerMatch ClubPointsPerMatchAvg,
        a.ClubNumPlayersUsedPerMatch ClubNumPlayersUsedPerMatchíAvg,
        a.ClubGoalPerShotOnTargetPerMatch ClubGoalPerShotOnTargetPerMatchAvg,
        a.ClubShotOnTargetAgainstUsedPerMatch ClubShotOnTargetAgainstUsedPerMatchAvg


        FROM predict_df DF 

        LEFT JOIN avg_df i
        ON DF.ClubInvolved = i.ClubInvolved
        AND DF.Season = i.Season

        LEFT JOIN avg_df_club a
        ON 'Villarreal CF'= a.Club
        AND DF.Season = a.Season
        """

predict_df = ps.sqldf(q1, locals())

predict_df = predict_df.drop_duplicates()

predict_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13102 entries, 0 to 13101
Columns: 132 entries, Name to ClubShotOnTargetAgainstUsedPerMatchAvg
dtypes: float64(63), int64(47), object(22)
memory usage: 13.3+ MB


Predict

In [566]:
# predict_df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/predict_df_temp3.csv")
# predict_df = predict_df.drop('Unnamed: 0', axis = 1)

team_df = predict_df[predict_df.ClubInvolved == 'Villarreal CF'][['ClubInvolvedWin', 'ClubInvolvedDraw', 'ClubInvolvedLoose', 'ClubInvolvedRank',\
    'ClubInvolvedAge', 'ClubInvolvedNumPlayersUsed', 'ClubInvolvedGoalPerShotOnTarget', 'ClubInvolvedPoints',\
        'ClubInvolvedShotOnTargetAgainst']].iloc[0]

predict_df['Pace_main'] = round(predict_df['Acceleration'] * 0.45 + predict_df['Sprint_speed'] * 0.55, 0)
predict_df['Shooting_main'] = round(predict_df['Finishing'] * 0.45 + predict_df['Long_shots'] * 0.20 + predict_df['Penalties'] * 0.05 + \
     predict_df['Positioning'] * 0.05 + predict_df['Shot_power'] * 0.20 + predict_df['Volleys'] * 0.05, 0)
predict_df['Dribbling_main'] = round(predict_df['Agility'] * 0.1 + predict_df['Balance'] * 0.05 + predict_df['Ball_control'] * 0.35 + \
     predict_df['Dribbling'] * 0.5, 0)
predict_df['Defending_main'] = round(predict_df['Heading_accuracy'] * 0.1 + predict_df['Interceptions'] * 0.2 + predict_df['Marking'] * 0.3 + \
     predict_df['Sliding_tackle'] * 0.3 + predict_df['Standing_tackle'] * 0.3, 0)
predict_df['Passing_main'] = round(predict_df['Crossing'] * 0.2 + predict_df['Curve'] * 0.05 + predict_df['FK_accuracy'] * 0.05 + \
     predict_df['Long_passing'] * 0.15 + predict_df['Short_passing'] * 0.35 + predict_df['Vision'] * 0.25, 0)
predict_df['Physical_main'] = round(predict_df['Aggression'] * 0.2 + predict_df['Jumping'] * 0.05 + predict_df['Stamina'] * 0.25 + \
     predict_df['Strength'] * 0.5, 0)

gk_predict = predict_df[predict_df.Position == 'GK']
outfielder_predict = predict_df[predict_df.Position != 'GK']

team_df['ClubInvolvedPlayed'] = team_df['ClubInvolvedWin'] + team_df['ClubInvolvedDraw'] + team_df['ClubInvolvedLoose']
outfielder_predict['ClubAge'] = team_df['ClubInvolvedAge']
outfielder_predict['ClubPointsPerMatch'] = team_df['ClubInvolvedPoints'] / team_df['ClubInvolvedPlayed']
outfielder_predict['ClubRank'] = team_df['ClubInvolvedRank']
outfielder_predict['ClubNumPlayersUsedPerMatch'] = team_df['ClubInvolvedNumPlayersUsed'] / team_df['ClubInvolvedPlayed']
outfielder_predict['ClubGoalPerShotOnTargetPerMatch'] = team_df['ClubInvolvedGoalPerShotOnTarget'] / team_df['ClubInvolvedPlayed']
outfielder_predict['ClubShotOnTargetAgainstUsedPerMatch'] = team_df['ClubInvolvedShotOnTargetAgainst'] / team_df['ClubInvolvedPlayed']

outfielder_predict = outfielder_predict.drop(['GK_diving', 'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes',\
    'ClubInvolvedSaves', 'ClubInvolvedSaveRate', 'GKMatchPlayed', 'GKStarts', 'GKMin', 'GK90s', 'GKGoalAgainst', 'GKGoalAgainstPer90',\
        'GKSaves', 'GKSavePercent', 'GKWin', 'GKDraw', 'GKLost', 'GKCleanSheet', 'GKCleanSheetPercent'], axis = 1)

outfielder_predict = outfielder_predict.drop(['ShotsOnTargetPercent', 'ClubInvolvedShotsOnTargetRate', 'ClubInvolvedGoalPerShot',\
    'ShotsPer90', 'Shots', 'ClubInvolvedShotsPer90', 'ClubInvolvedShots'], axis = 1)

outfielder_predict = outfielder_predict.drop(['GoalsPerShot', 'ClubInovlvedPossession'], axis = 1)

outfielder_predict['ClubInvolvedPlayed'] = outfielder_predict['ClubInvolvedWin'] + outfielder_predict['ClubInvolvedDraw'] + outfielder_predict['ClubInvolvedLoose']

outfielder_predict['ClubInvolvedWinPerMatch'] = outfielder_predict['ClubInvolvedWin'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedDrawPerMatch'] = outfielder_predict['ClubInvolvedDraw'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedLoosePerMatch'] = outfielder_predict['ClubInvolvedLoose'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalForPerMatch'] = outfielder_predict['ClubInvolvedGoalFor'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalAgainstPerMatch'] = outfielder_predict['ClubInvolvedGoalAgainst'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedPointsPerMatch'] = outfielder_predict['ClubInvolvedPoints'] / outfielder_predict['ClubInvolvedPlayed']


outfielder_predict['ClubInvolvedAssistsPerMatch'] = outfielder_predict['ClubInvolvedAssists'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedNonPenaltyGoalPerMatch'] = outfielder_predict['ClubInvolvedNonPenaltyGoal'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedYellowCardPerMatch'] = outfielder_predict['ClubInvolvedYellowCard'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedRedCardPerMatch'] = outfielder_predict['ClubInvolvedRedCard'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedNumPlayersUsedPerMatch'] = outfielder_predict['ClubInvolvedNumPlayersUsed'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedShotsOnTargetPerMatch'] = outfielder_predict['ClubInvolvedShotsOnTarget'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalPerShotOnTargetPerMatch'] = outfielder_predict['ClubInvolvedGoalPerShotOnTarget'] / outfielder_predict['ClubInvolvedPlayed']


outfielder_predict['ClubInvolvedShotOnTargetAgainstUsedPerMatch'] = outfielder_predict['ClubInvolvedShotOnTargetAgainst'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedCleanSheetsPerMatch'] = outfielder_predict['ClubInvolvedCleanSheets'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['MatchesPlayedPerClubMatches'] = outfielder_predict['MatchesPlayed'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['StartsPerClubMatches'] = outfielder_predict['Starts'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['NonPenaltyGoalsPerMinutes'] = outfielder_predict['MatchesPlayed'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['GoalsPerMinutesPlayed'] = outfielder_predict['Goals'] / outfielder_predict['MinutesPlayed']
outfielder_predict['AssistsPerMinutesPlayed'] = outfielder_predict['Assists'] / outfielder_predict['MinutesPlayed']
outfielder_predict['NonPenaltyGoalsPerMinutesPlayed'] = outfielder_predict['NonPenaltyGoals'] / outfielder_predict['MinutesPlayed']
outfielder_predict['YellowCardsPerMinutesPlayed'] = outfielder_predict['YellowCards'] / outfielder_predict['MinutesPlayed']
outfielder_predict['RedCardsPerMinutesPlayed'] = outfielder_predict['RedCards'] / outfielder_predict['MinutesPlayed']
outfielder_predict['ShotsOnTargetPerMinutesPlayed'] = outfielder_predict['ShotsOnTarget'] / outfielder_predict['MinutesPlayed']

outfielder_predict = outfielder_predict.drop(['ClubInvolvedPlayed', 'ClubInvolvedWin', 'ClubInvolvedDraw',\
     'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints',\
               'ClubInvolvedAssists', 'ClubInvolvedNonPenaltyGoal', 'ClubInvolvedYellowCard', 'ClubInvolvedRedCard',\
                     'ClubInvolvedNumPlayersUsed',\
                         'ClubInvolvedShotsOnTarget', 'ClubInvolvedGoalPerShotOnTarget', 'ClubInvolvedShotOnTargetAgainst', 'ClubInvolvedCleanSheets',\
                              'MatchesPlayed', 'Starts', 'MatchesPlayed', 'Goals', 'Assists', 'NonPenaltyGoals', 'YellowCards',\
                                   'RedCards', 'ShotsOnTarget',  'GoalsPer90', 'AssistsPer90', 'GoalsAssistsPer90', 'ShotsOnTargetPer90'], axis=1)

outfielder_predict['CorrMarketValue'] = outfielder_predict['NewMarketValue']

result_df = outfielder_predict[['Name', 'ClubInvolved', 'MainPosition',  'ID', 'Nationality']]

outfielder_predict = outfielder_predict.drop([ 'Name', 'ContractGood', 'Player',\
    'Nation', 'FbrefClub', 'GoalPerShotOnTarget', 'Pos', 'MarketValue', 'Composure', 'ID', 'Joined', 'NewMarketValue', 'Nationality'], axis=1)
    #'ClubInvolved',

outfielder_predict = outfielder_predict[feature_list]

/var/folders/8m/8q13t9cn0917nvwcf017zkbc0000gn/T/ipykernel_1519/3192550910.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outfielder_predict['ClubAge'] = team_df['ClubInvolvedAge']
/var/folders/8m/8q13t9cn0917nvwcf017zkbc0000gn/T/ipykernel_1519/3192550910.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outfielder_predict['ClubPointsPerMatch'] = team_df['ClubInvolvedPoints'] / team_df['ClubInvolvedPlayed']
/var/folders/8m/8q13t9cn0917nvwcf017zkbc0000gn/T/ipykernel_1519/3192550910.py:26: SettingWit

In [567]:
#encode
outfielder_predict_cbe = cbe_encoder.transform(outfielder_predict)

In [568]:
#impute
outfielder_predict_cbe_imp = imputer.transform(outfielder_predict_cbe)
outfielder_predict_cbe_imp = pd.DataFrame(data = outfielder_predict_cbe_imp, columns = outfielder_predict_cbe.columns)

In [569]:
#standardization
outfielder_predict_imp_st = scaler.transform(outfielder_predict_cbe_imp)
outfielder_predict_imp_st = pd.DataFrame(data = outfielder_predict_imp_st, columns = outfielder_predict_cbe_imp.columns)

In [570]:
outfielder_predict_imp_st['LeagueInvolved_CorrMarketValue'] = outfielder_predict_imp_st['LeagueInvolved'] * outfielder_predict_imp_st['CorrMarketValue']
outfielder_predict_imp_st['LeagueInvolved_MinutesPlayedPer90'] = outfielder_predict_imp_st['LeagueInvolved'] * outfielder_predict_imp_st['MinutesPlayedPer90']
outfielder_predict_imp_st['LeagueInvolved_Season'] = outfielder_predict_imp_st['LeagueInvolved'] / outfielder_predict_imp_st['Season']
outfielder_predict_imp_st['Season_SP_LA_LIGA'] = outfielder_predict_imp_st['Season'] / outfielder_predict_imp_st['SP_LA_LIGA']

In [373]:
# define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)

In [571]:
#predict 
model.fit(X_res , y_res)
result_df['confidence'] = model.predict_proba(outfielder_predict_imp_st)[:, 1]
result_df.rename(columns={'ClubInvolved': 'Team', 'MainPosition':'Position', 'confidence':'Probability'}, inplace=True)

result_df.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/TransferPredict/transfer_predict_api/transferPredict/transferPredictApp/Predictions/Data/result_SP_2p.csv")


result_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/result_SP_2p.xlsx")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [491]:
result_df.head()

,Name,Team,Position,ID,Nationality,Probability
0,L. Messi,FC Barcelona,Forward,158023,Argentina,0.079769
1,Cristiano Ronaldo,Juventus,Forward,20801,Portugal,0.008754
3,K. De Bruyne,Manchester City,Midfielder,192985,Belgium,0.002139
4,Neymar Jr,Paris Saint-Germain,Forward,190871,Brazil,0.032634
5,R. Lewandowski,FC Bayern München,Forward,188545,Poland,0.003840


In [572]:
len(result_df[(result_df.Name.isin(['R. De Paul',
'Matheus Cunha',
'Emerson',
'Eric García',
'S. Agüero',
'M. Depay',
'E. Camavinga',
'D. Alaba',
'Rafa Mir',
'T. Delaney',
'L. Augustinsson',
'E. Lamela',
'Diego Rico',
'A. Danjuma Groeneveld',
'J. Foyth',
'B. Dia',
'Manu Morlanes',
'A. Mandi',
'Y. Sabaly',
'Rochina',
'Monchu',
'Escudero',
'Javi Galán',
'F. Cervi']))&(result_df.Probability>=0.5)])
# 'Rubén Sobrino',
# 'L. Dimata',
# 'Vadillo',
# 'Sergi Gómez',
# 'Miguelón',
# 'Aleix Vidal',
# 'A. Budimir',
# 'Ramalho',
# 'Cote',
# 'Kike García',
# 'Jesús Areso',
# 'Franquesa',
# 'Marcos André',
# 'J. Silva',
# 'S. Mitrović',
# 'M. Sylla',
# 'Omar Mascarell',
# 'Gumbau',
# 'Bigas',
# 'M. Hoppe',
# 'Kangin Lee',
# 'Jaume Costa',
# 'Fran García',
# 'R. Nteka',
# 'Falcao',
# 'I. Balliu']))&(result_df.Probability>=0.5)])

12